# Unity Catalog and Table Management Lab - Solution Guide

## Overview
This solution guide provides comprehensive answers and explanations for all exercises in the Unity Catalog and Table Management lab. Each solution includes detailed explanations and best practices.

---

## Exercise 1 Solutions: Unity Catalog Setup and Managed vs External Tables

### Questions & Answers

**Q1: Where is the data for the managed table stored?**

**Answer:**
Managed tables in Unity Catalog store their data in **Unity Catalog managed storage locations**:
- **AWS**: `s3://unity-catalog-storage/metastore-id/tables/catalog.schema.table/`
- **Azure**: `abfss://container@storage.dfs.core.windows.net/metastore-id/tables/catalog.schema.table/`
- **GCP**: `gs://unity-catalog-storage/metastore-id/tables/catalog.schema.table/`

The exact location is managed automatically by Unity Catalog and can be viewed using:
```sql
DESCRIBE DETAIL hr_analytics_lab.hr_data.employees_managed;
-- Look for the 'location' field in the output
```

**Storage characteristics:**
- **Fully managed** by Databricks Unity Catalog
- **Automatic organization** by catalog/schema/table hierarchy
- **No user configuration** of storage paths required
- **Integrated backup** and versioning through Unity Catalog

**Q2: What are the benefits of using managed tables?**

**Answer:**
Key benefits of managed tables:

1. **Simplified Management:**
   - No need to manage storage paths manually
   - Automatic file organization and cleanup
   - Built-in metadata management

2. **Enhanced Security:**
   - Centralized access control through Unity Catalog
   - Automatic encryption in transit and at rest
   - No direct cloud storage permissions required

3. **Data Governance:**
   - Automatic lineage tracking
   - Built-in audit logging
   - Centralized metadata and tagging

4. **Performance Optimization:**
   - Optimized storage layout
   - Automatic compaction and optimization
   - Better query performance through Unity Catalog optimizations

5. **Backup & Recovery:**
   - Automatic backup integration
   - Point-in-time recovery capabilities
   - Disaster recovery built-in

**Q3: Who owns the lifecycle of managed table data?**

**Answer:**
**Unity Catalog owns the complete lifecycle** of managed table data:

**Creation Phase:**
- Unity Catalog creates storage directories
- Manages initial table structure and metadata
- Establishes security and access controls

**Operational Phase:**
- Handles data file organization and compaction
- Manages metadata updates and schema evolution  
- Controls access permissions and audit logging

**Maintenance Phase:**
- Automatic cleanup of old files (via VACUUM)
- Optimization and performance tuning
- Backup and replication management

**Deletion Phase:**
- When table is dropped, Unity Catalog removes ALL data
- Complete cleanup of storage and metadata
- No orphaned files left behind

**User Responsibilities:**
- Data insertion and business logic
- Query performance optimization 
- Schema design and constraint definition
- Access control configuration

---

## Exercise 2 Solutions: External Locations and Storage Credentials

### Questions & Answers

**Q1: What are the main benefits of external locations in Unity Catalog?**

**Answer:**
External locations provide several critical advantages:

1. **Data Sovereignty:**
   ```sql
   -- Data remains in your controlled cloud storage
   CREATE EXTERNAL LOCATION my_data_lake
   URL 's3://my-company-datalake/sensitive-data/'
   CREDENTIAL my_aws_credential;
   ```
   - **Ownership**: You maintain full ownership of data
   - **Control**: Complete control over storage policies
   - **Compliance**: Meet regulatory requirements for data residency

2. **Integration with Existing Systems:**
   ```sql
   -- Connect to existing data lakes
   CREATE TABLE analytics.sales_external
   USING DELTA
   LOCATION 's3://existing-datalake/sales/'
   ```
   - **Legacy Integration**: Connect existing data lakes to Unity Catalog
   - **Multi-tool Access**: Same data accessible from multiple analytics tools
   - **Gradual Migration**: Migrate to Unity Catalog without data movement

3. **Advanced Security:**
   ```sql
   CREATE STORAGE CREDENTIAL secure_credential
   TYPE IAM
   IAM_ROLE 'arn:aws:iam::account:role/databricks-restricted-role'
   ```
   - **Fine-grained Access**: Credential-based access control
   - **Network Security**: VPC endpoints and network isolation
   - **Audit Trails**: Complete access logging

4. **Cost Management:**
   - **Storage Flexibility**: Use different storage classes (hot, cold, archive)
   - **Lifecycle Policies**: Automatic data archiving
   - **Multi-region**: Optimize for regional access patterns

5. **Disaster Recovery:**
   - **Geographic Distribution**: Data in multiple regions
   - **Backup Strategies**: Independent backup systems
   - **Business Continuity**: Data survives platform changes

**Q2: When would you choose external tables over managed tables?**

**Answer:**
Choose external tables when:

**Regulatory & Compliance Scenarios:**
```sql
-- Financial data that must remain in specific regions
CREATE TABLE finance.transactions_external
LOCATION 's3://financial-data-eu-west-1/transactions/'
```
- **Data residency** requirements (GDPR, financial regulations)
- **Audit requirements** for data location control
- **Compliance** with industry standards

**Legacy Integration:**
```sql
-- Existing data lake with multiple consumers
CREATE TABLE marketing.customer_data_external  
LOCATION 's3://existing-customer-lake/profiles/'
```
- **Existing data lakes** with established processes
- **Multi-platform access** (Spark, Snowflake, etc.)
- **Gradual migration** strategies

**Advanced Architecture Patterns:**
```sql
-- Data mesh pattern with domain ownership
CREATE TABLE sales.regional_sales_external
LOCATION 's3://sales-domain-storage/regional/'
```
- **Data mesh** architectures
- **Domain-driven** data ownership
- **Federated** data management

**Performance & Scale:**
- **Very large datasets** (petabyte scale)
- **Specific storage optimizations** required
- **Custom partitioning** strategies

**Use managed tables when:**
- **Simple analytics** use cases
- **Standard compliance** requirements sufficient
- **Want simplified management**
- **Team lacks cloud storage expertise**

**Q3: What security advantages do storage credentials provide?**

**Answer:**
Storage credentials provide multi-layered security:

1. **Identity-Based Access Control:**
   ```sql
   CREATE STORAGE CREDENTIAL analytics_team_credential
   TYPE IAM
   IAM_ROLE 'arn:aws:iam::123456789:role/analytics-team-role'
   COMMENT 'Restricted access for analytics team'
   ```
   - **IAM Integration**: Leverages cloud provider IAM systems
   - **Principle of Least Privilege**: Minimal required permissions
   - **Role-Based Access**: Team and function-specific credentials

2. **Centralized Credential Management:**
   ```sql
   -- Centralized credential updates
   ALTER STORAGE CREDENTIAL analytics_team_credential
   SET IAM_ROLE 'arn:aws:iam::123456789:role/analytics-team-role-v2'
   ```
   - **Single Point of Control**: Update credentials centrally
   - **Audit Trail**: Track all credential usage
   - **Rotation Policies**: Regular credential rotation

3. **Fine-Grained Access Control:**
   ```sql
   -- Grant specific access to external locations
   GRANT CREATE TABLE ON EXTERNAL LOCATION secure_data_location 
   TO analytics_team
   ```
   - **Location-Level Permissions**: Control access per storage location
   - **Operation-Level Control**: Read vs write vs admin permissions
   - **Resource Isolation**: Separate credentials for different data tiers

4. **Network Security:**
   ```sql
   CREATE EXTERNAL LOCATION vpc_only_data
   URL 's3://private-vpc-bucket/sensitive/'
   CREDENTIAL vpc_restricted_credential
   ```
   - **VPC Endpoints**: Private network connectivity
   - **Network Policies**: Restrict network access
   - **Encryption in Transit**: Secure data transfer

5. **Monitoring & Compliance:**
   - **Access Logging**: Complete audit trail of data access
   - **Usage Monitoring**: Track credential usage patterns
   - **Compliance Reporting**: Generate access reports for audits

**Security Best Practices:**
```sql
-- Example comprehensive security setup
CREATE STORAGE CREDENTIAL production_secure_credential
TYPE IAM
IAM_ROLE 'arn:aws:iam::prod:role/databricks-production-readonly'
COMMENT 'Production data access with read-only permissions'

CREATE EXTERNAL LOCATION production_data_lake
URL 's3://prod-data-lake-encrypted/analytics/'
CREDENTIAL production_secure_credential
COMMENT 'Production data lake with encryption and VPC access'
```

---

## Exercise 3 Solutions: Working with Different Types of Views

### Questions & Answers

**Q1: What is the difference between temporary and global temporary views?**

**Answer:**
Key differences between temporary and global temporary views:

| Aspect | Temporary Views | Global Temporary Views |
|--------|----------------|----------------------|
| **Scope** | Current session only | Cross-session within cluster |
| **Syntax** | `CREATE TEMPORARY VIEW` | `CREATE GLOBAL TEMPORARY VIEW` |
| **Namespace** | Default namespace | `global_temp` database |
| **Lifetime** | Until session ends | Until cluster terminates |
| **Sharing** | Single user/session | Multiple sessions on same cluster |

**Temporary Views:**
```sql
-- Only visible in current session
CREATE TEMPORARY VIEW session_analytics AS
SELECT department, AVG(salary) as avg_sal
FROM employees WHERE is_active = true
GROUP BY department;

-- Query in same session: ✅ Works
SELECT * FROM session_analytics;

-- Query from different session: ❌ Fails - view not found
```

**Global Temporary Views:**
```sql
-- Visible across sessions on same cluster
CREATE GLOBAL TEMPORARY VIEW global_temp.cluster_analytics AS
SELECT department, COUNT(*) as emp_count
FROM employees WHERE is_active = true
GROUP BY department;

-- Query from any session on cluster: ✅ Works
SELECT * FROM global_temp.cluster_analytics;
```

**Use Cases:**
- **Temporary Views**: Personal analysis, debugging, ad-hoc queries
- **Global Temporary Views**: Cross-session collaboration, shared calculations

**Q2: When would you use a stored view vs a temporary view?**

**Answer:**
Choose based on persistence, sharing, and governance needs:

**Use Stored Views When:**

1. **Business Logic Standardization:**
   ```sql
   CREATE VIEW finance.monthly_revenue AS
   SELECT 
       YEAR(order_date) as year,
       MONTH(order_date) as month,
       SUM(amount) as total_revenue
   FROM sales.orders
   GROUP BY YEAR(order_date), MONTH(order_date)
   ```
   - **Reusable** across multiple queries and users
   - **Consistent** business logic implementation
   - **Centralized** definition management

2. **Security & Access Control:**
   ```sql
   CREATE VIEW hr.employee_summary AS
   SELECT employee_id, first_name, last_name, department
   FROM hr.employees_full  -- Hide salary and PII
   WHERE is_active = true
   ```
   - **Data masking** and column-level security
   - **Row-level filtering** for different user groups
   - **Governance** through Unity Catalog permissions

3. **Performance Optimization:**
   ```sql
   CREATE VIEW analytics.customer_360 AS
   SELECT c.*, o.total_orders, p.total_payments
   FROM customers c
   JOIN (SELECT customer_id, COUNT(*) as total_orders FROM orders GROUP BY customer_id) o
   JOIN (SELECT customer_id, SUM(amount) as total_payments FROM payments GROUP BY customer_id) p
   ```
   - **Complex joins** abstracted into simple interface
   - **Query optimization** by database engine
   - **Consistent** performance patterns

**Use Temporary Views When:**

1. **Exploratory Data Analysis:**
   ```sql
   CREATE TEMPORARY VIEW temp_outliers AS
   SELECT * FROM sales_data 
   WHERE amount > (SELECT PERCENTILE_APPROX(amount, 0.95) FROM sales_data)
   ```
   - **Ad-hoc analysis** and data exploration
   - **Quick prototyping** of analytical queries
   - **Personal workspace** without affecting others

2. **ETL Pipeline Intermediate Steps:**
   ```sql
   -- Step 1: Clean data
   CREATE TEMPORARY VIEW clean_data AS
   SELECT * FROM raw_data WHERE quality_flag = 'valid'
   
   -- Step 2: Transform
   CREATE TEMPORARY VIEW transformed_data AS  
   SELECT id, UPPER(name), normalized_amount FROM clean_data
   
   -- Step 3: Load final table
   INSERT INTO final_table SELECT * FROM transformed_data
   ```
   - **Pipeline isolation** from concurrent executions
   - **Memory efficiency** (automatic cleanup)
   - **No metadata pollution**

3. **Session-Specific Configuration:**
   ```sql
   CREATE TEMPORARY VIEW current_user_data AS
   SELECT * FROM all_user_data 
   WHERE user_id = '${current_user_id}'
   ```
   - **User-specific** data filtering
   - **Dynamic** content based on session context
   - **Privacy** (no persistent access logs)

**Decision Matrix:**
- **Permanent + Shared**: Stored View
- **Permanent + Personal**: Consider materialized table instead
- **Temporary + Shared**: Global Temporary View
- **Temporary + Personal**: Temporary View

**Q3: How do views contribute to data governance in Unity Catalog?**

**Answer:**
Views are critical governance tools in Unity Catalog:

1. **Access Control Layer:**
   ```sql
   -- Create view that masks PII
   CREATE VIEW hr.employees_public AS
   SELECT 
       employee_id,
       first_name,
       last_name,
       department,
       CASE 
           WHEN current_user() IN ('hr_manager', 'ceo') THEN salary
           ELSE NULL 
       END as salary
   FROM hr.employees_full
   
   -- Grant access to view, not base table
   GRANT SELECT ON VIEW hr.employees_public TO analysts
   ```
   - **Column-level security**: Hide sensitive columns
   - **Row-level filtering**: Show only relevant records
   - **Dynamic access**: User-based data access

2. **Data Lineage and Audit:**
   ```sql
   CREATE VIEW analytics.quarterly_metrics AS
   SELECT quarter, region, SUM(revenue) as total_revenue
   FROM sales.fact_sales  -- Lineage tracked automatically
   GROUP BY quarter, region
   ```
   - **Automatic lineage**: Unity Catalog tracks view dependencies
   - **Impact analysis**: Understand downstream effects of changes
   - **Audit trails**: Track view usage and access patterns

3. **Data Quality Enforcement:**
   ```sql
   CREATE VIEW finance.validated_transactions AS
   SELECT * FROM finance.all_transactions
   WHERE amount > 0 
     AND transaction_date >= '2020-01-01'
     AND status IN ('completed', 'pending')
   ```
   - **Quality filters**: Ensure only valid data is accessible
   - **Business rules**: Encode domain knowledge in views
   - **Consistency**: Standardize data quality across consumers

4. **Documentation and Metadata:**
   ```sql
   CREATE VIEW customer.customer_metrics
   COMMENT 'Customer lifetime value and engagement metrics. Updated daily at 2 AM UTC.'
   AS SELECT customer_id, clv, engagement_score FROM customer.analytics_base
   
   ALTER VIEW customer.customer_metrics 
   SET TBLPROPERTIES (
     'owner' = 'customer_analytics_team',
     'refresh_frequency' = 'daily',
     'data_classification' = 'customer_pii'
   )
   ```
   - **Self-documenting**: Comments explain business context
   - **Metadata tags**: Classification and ownership information
   - **Discovery**: Help users find and understand data

5. **Change Management:**
   ```sql
   -- Version 1: Original view
   CREATE VIEW sales.monthly_summary AS
   SELECT month, SUM(amount) as total FROM sales.orders GROUP BY month
   
   -- Version 2: Enhanced view (backward compatible)
   CREATE OR REPLACE VIEW sales.monthly_summary AS
   SELECT 
       month, 
       SUM(amount) as total,
       COUNT(*) as transaction_count,  -- New column
       AVG(amount) as avg_amount       -- New column
   FROM sales.orders GROUP BY month
   ```
   - **Schema evolution**: Add columns without breaking consumers
   - **Backward compatibility**: Maintain existing interfaces
   - **Centralized updates**: Change logic in one place

**Governance Best Practices:**
- **Naming conventions**: Use consistent, descriptive view names
- **Documentation**: Always add meaningful comments
- **Access patterns**: Grant least-privilege access
- **Regular review**: Audit view usage and permissions
- **Version control**: Track view definition changes

---

## Exercise 4 Solutions: CREATE TABLE AS SELECT (CTAS) Operations

### Questions & Answers

**Q1: How does CTAS differ from regular CREATE TABLE followed by INSERT?**

**Answer:**
CTAS and CREATE + INSERT have significant differences:

**CREATE TABLE AS SELECT (CTAS):**
```sql
-- Single atomic operation
CREATE TABLE analytics.high_performers AS
SELECT employee_id, salary, department
FROM hr.employees WHERE salary > 100000
```

**CREATE TABLE + INSERT:**
```sql
-- Two separate operations
CREATE TABLE analytics.high_performers (
    employee_id INT,
    salary DOUBLE, 
    department STRING
);

INSERT INTO analytics.high_performers
SELECT employee_id, salary, department  
FROM hr.employees WHERE salary > 100000
```

**Key Differences:**

1. **Atomicity:**
   - **CTAS**: Single atomic transaction - either complete success or complete failure
   - **CREATE + INSERT**: Two transactions - table might exist empty if INSERT fails

2. **Schema Management:**
   - **CTAS**: Schema automatically inferred from SELECT query
   - **CREATE + INSERT**: Schema must be explicitly defined and match INSERT data

3. **Performance:**
   - **CTAS**: Optimized single-pass operation, parallel execution
   - **CREATE + INSERT**: Potential overhead from two operations, metadata updates

4. **Metadata Handling:**
   - **CTAS**: Column names, types, nullability inferred automatically
   - **CREATE + INSERT**: Manual schema definition, potential type mismatches

5. **Error Handling:**
   ```sql
   -- CTAS: If query fails, no table created
   CREATE TABLE test AS SELECT bad_column FROM non_existent_table  -- Clean failure
   
   -- CREATE + INSERT: Table exists even if INSERT fails
   CREATE TABLE test (id INT)        -- ✅ Succeeds
   INSERT INTO test SELECT bad_col   -- ❌ Fails, but table still exists
   ```

**When to Use Each:**

**Use CTAS when:**
- Creating tables from analytical queries
- Schema should match source data exactly
- Want atomic operation guarantees
- Prototyping and exploration

**Use CREATE + INSERT when:**
- Need explicit schema control
- Adding constraints not supported in CTAS
- Incremental loading patterns
- Complex business logic requiring separation

**Q2: What are the benefits of partitioning in the CTAS example?**

**Answer:**
The CTAS partitioning example demonstrates several key benefits:

```sql
CREATE TABLE hr_analytics_lab.analytics.employees_by_location
COMMENT "Contains PII - Employee data partitioned by city and birth_year"
PARTITIONED BY (city, birth_year)
AS
SELECT 
    employee_id as id,
    name,
    email,
    birth_date,
    city,
    YEAR(birth_date) as birth_year,
    department,
    salary
FROM employees WHERE is_active = true
```

**Performance Benefits:**

1. **Query Performance:**
   ```sql
   -- Partition elimination - only scans relevant partitions
   SELECT COUNT(*) FROM employees_by_location 
   WHERE city = 'New York' AND birth_year = 1985
   -- Only scans city=New York/birth_year=1985 partition
   ```
   - **Partition elimination**: Skip irrelevant data files
   - **Parallel processing**: Each partition processed independently
   - **Reduced I/O**: Read only necessary data

2. **Data Organization:**
   ```
   /employees_by_location/
   ├── city=New York/birth_year=1985/part-001.parquet
   ├── city=New York/birth_year=1986/part-002.parquet  
   ├── city=Chicago/birth_year=1985/part-003.parquet
   └── city=Chicago/birth_year=1986/part-004.parquet
   ```
   - **Logical organization**: Data organized by business dimensions
   - **File pruning**: Query engines skip entire directories
   - **Metadata efficiency**: Partition metadata cached separately

**Operational Benefits:**

3. **Data Management:**
   ```sql
   -- Drop specific partitions efficiently
   ALTER TABLE employees_by_location 
   DROP PARTITION (city = 'Chicago', birth_year = 1980)
   
   -- Add new partition
   ALTER TABLE employees_by_location 
   ADD PARTITION (city = 'Austin', birth_year = 2000)
   ```
   - **Granular operations**: Manage subsets of data efficiently
   - **Bulk operations**: Load/delete entire partitions quickly
   - **Maintenance**: OPTIMIZE and VACUUM per partition

4. **Compliance and Retention:**
   ```sql
   -- Delete old data by partition (GDPR right to be forgotten)
   DELETE FROM employees_by_location 
   WHERE city = 'Paris' AND birth_year < 1970
   ```
   - **Data retention**: Delete old partitions based on business rules
   - **Compliance**: Remove data by geographic region
   - **Archival**: Move old partitions to cold storage

**Considerations:**

5. **Partition Design Best Practices:**
   ```sql
   -- Good partitioning: balanced, query-aligned
   PARTITIONED BY (region, year)  -- ~10-100 partitions each
   
   -- Poor partitioning: too granular
   PARTITIONED BY (employee_id)   -- Thousands of tiny partitions
   
   -- Poor partitioning: too broad  
   PARTITIONED BY (country)       -- Only 3-5 large partitions
   ```
   - **Cardinality balance**: 10-1000 partitions optimal
   - **Query alignment**: Partition keys match common filter patterns
   - **Data size**: Each partition 100MB-1GB ideal

**Q3: When would you use CTAS vs creating views?**

**Answer:**
Choose based on performance, complexity, and update patterns:

**Use CTAS (Materialized Data) When:**

1. **Performance-Critical Scenarios:**
   ```sql
   -- Heavy aggregation that's queried frequently
   CREATE TABLE analytics.daily_revenue_summary AS
   SELECT 
       DATE(order_timestamp) as order_date,
       product_category,
       SUM(revenue) as total_revenue,
       COUNT(*) as order_count,
       AVG(revenue) as avg_order_value
   FROM sales.orders o
   JOIN products.catalog p ON o.product_id = p.id
   WHERE order_timestamp >= '2023-01-01'
   GROUP BY DATE(order_timestamp), product_category
   ```
   **Benefits:**
   - **Fast queries**: Pre-computed results, no aggregation at query time
   - **Consistent performance**: Query time independent of source data size
   - **Resource efficiency**: No repeated expensive computations

2. **Complex Transformations:**
   ```sql
   -- Complex business logic that's expensive to compute
   CREATE TABLE customer.loyalty_segments AS
   WITH customer_metrics AS (
     SELECT customer_id,
            COUNT(*) as total_orders,
            SUM(amount) as lifetime_value,
            AVG(DATEDIFF(order_date, LAG(order_date) OVER (PARTITION BY customer_id ORDER BY order_date))) as avg_days_between_orders
     FROM orders GROUP BY customer_id
   ),
   rfm_analysis AS (
     SELECT customer_id,
            NTILE(5) OVER (ORDER BY recency) as recency_score,
            NTILE(5) OVER (ORDER BY frequency) as frequency_score,
            NTILE(5) OVER (ORDER BY monetary) as monetary_score
     FROM customer_metrics
   )
   SELECT customer_id, recency_score + frequency_score + monetary_score as loyalty_score
   FROM rfm_analysis
   ```
   **Benefits:**
   - **Complexity isolation**: Hide complex logic from end users
   - **Debugging**: Intermediate results available for validation
   - **Reliability**: Pre-validated transformations

3. **Data Stability:**
   ```sql
   -- Snapshot historical data that changes over time
   CREATE TABLE reporting.monthly_snapshot_202310 AS
   SELECT customer_id, account_balance, credit_score, risk_category
   FROM finance.customer_profiles
   WHERE snapshot_date = '2023-10-31'
   ```
   **Benefits:**
   - **Point-in-time consistency**: Results don't change with source data updates
   - **Historical preservation**: Maintain historical snapshots
   - **Audit compliance**: Fixed data for reporting periods

**Use Views When:**

1. **Real-Time Requirements:**
   ```sql
   CREATE VIEW operations.current_system_status AS
   SELECT 
       system_component,
       status,
       last_heartbeat,
       CASE 
         WHEN last_heartbeat < CURRENT_TIMESTAMP() - INTERVAL 5 MINUTES 
         THEN 'ALERT' ELSE 'OK' 
       END as health_status
   FROM monitoring.heartbeats
   ```
   **Benefits:**
   - **Always current**: Reflects latest data state
   - **No maintenance**: Automatically updates with source changes
   - **Real-time insights**: Critical for operational dashboards

2. **Security and Access Control:**
   ```sql
   CREATE VIEW finance.user_accessible_accounts AS
   SELECT account_id, account_name, balance
   FROM finance.all_accounts a
   JOIN security.user_permissions p ON a.account_id = p.account_id
   WHERE p.user_id = current_user()
   ```
   **Benefits:**
   - **Dynamic security**: Access based on current user context
   - **No data duplication**: Single source of truth
   - **Governance**: Centralized access control logic

3. **Simple Transformations:**
   ```sql
   CREATE VIEW sales.readable_orders AS
   SELECT 
       order_id,
       customer_name,
       order_amount,
       order_date,
       CASE status_code 
         WHEN 'P' THEN 'Pending'
         WHEN 'S' THEN 'Shipped' 
         WHEN 'C' THEN 'Completed'
       END as order_status
   FROM sales.orders_raw
   ```
   **Benefits:**
   - **Low overhead**: Simple transformations don't justify materialization
   - **Always synchronized**: No risk of stale data
   - **Storage efficient**: No additional storage required

**Decision Framework:**
- **High query frequency + expensive computation** → CTAS
- **Real-time requirements + simple logic** → View  
- **Historical snapshots + compliance** → CTAS
- **Security layer + access control** → View
- **Complex analytics + performance critical** → CTAS
- **Data exploration + prototyping** → View

---

## Exercise 5 Solutions: Delta Lake Constraints Implementation

### Questions & Answers

**Q1: What is the difference between enforced constraints and metadata-only constraints?**

**Answer:**
Delta Lake constraints fall into two categories based on enforcement:

**Enforced Constraints (Runtime Enforcement):**

These are **actively validated by the Delta Engine at write time**:

1. **NOT NULL Constraints:**
   ```sql
   ALTER TABLE employees ADD CONSTRAINT emp_id_not_null NOT NULL (employee_id)
   ```
   **Enforcement Details:**
   - **WHO**: Delta Engine validates each row during write operations
   - **WHEN**: At write time (INSERT, UPDATE, MERGE, streaming writes)
   - **RESULT**: Write operation fails if constraint violated
   - **PERFORMANCE**: Minimal overhead - simple null checks

2. **CHECK Constraints:**
   ```sql
   ALTER TABLE employees ADD CONSTRAINT salary_positive CHECK (salary > 0)
   ALTER TABLE employees ADD CONSTRAINT valid_email CHECK (email LIKE '%@%.%')
   ```
   **Enforcement Details:**
   - **WHO**: Delta Engine evaluates CHECK expression for each row
   - **WHEN**: At write time for all write operations  
   - **RESULT**: Transaction fails if any row violates constraint
   - **PERFORMANCE**: Depends on complexity of CHECK expression

**Write-Time Validation Example:**
```python
# This will fail due to constraint violation
df = spark.createDataFrame([(1, None, -50000)], ['id', 'name', 'salary'])
df.write.format('delta').mode('append').saveAsTable('employees')
# Error: NOT NULL constraint violation (name) and CHECK constraint violation (salary)
```

**Metadata-Only Constraints (Informational):**

These are **stored as metadata** but **not enforced by Delta Engine**:

1. **PRIMARY KEY:**
   ```sql
   ALTER TABLE employees ADD CONSTRAINT pk_employee PRIMARY KEY (employee_id)
   ```

2. **FOREIGN KEY:**
   ```sql
   ALTER TABLE orders ADD CONSTRAINT fk_customer 
   FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
   ```

3. **UNIQUE:**
   ```sql
   ALTER TABLE employees ADD CONSTRAINT unique_email UNIQUE (email)
   ```

**Metadata-Only Details:**
- **WHO**: Unity Catalog stores constraint metadata
- **WHEN**: Information available for query optimization and governance
- **ENFORCEMENT**: None - duplicate/invalid data can be inserted
- **PURPOSE**: Documentation, query optimization hints, governance

**Constraint Enforcement Comparison:**

| Constraint Type | Enforced? | Performance Impact | Use Case |
|----------------|-----------|-------------------|----------|
| **NOT NULL** | ✅ Yes | Minimal | Data quality |
| **CHECK** | ✅ Yes | Low-Medium | Business rules |
| **PRIMARY KEY** | ❌ No | None | Documentation |
| **FOREIGN KEY** | ❌ No | None | Lineage |
| **UNIQUE** | ❌ No | None | Documentation |
| **DEFAULT** | ⚠️ Limited | None | SQL INSERT only |

**Q2: Why are CHECK constraints enforced at write time by the Delta Engine?**

**Answer:**
CHECK constraints are enforced at write time for several architectural and performance reasons:

1. **ACID Transaction Guarantees:**
   ```python
   # All-or-nothing approach
   try:
       df.write.format('delta').mode('append').saveAsTable('employees')
       # If ANY row violates CHECK constraint, ENTIRE batch fails
       print("All data written successfully")
   except Exception as e:
       # No partial writes - table remains in consistent state
       print(f"No data written due to constraint violation: {e}")
   ```
   **Benefits:**
   - **Atomicity**: Either all data passes validation or none is written
   - **Consistency**: Table never enters invalid state
   - **Data integrity**: Guaranteed constraint compliance

2. **Performance Optimization:**
   ```sql
   -- Constraint checked once during write
   ALTER TABLE sales ADD CONSTRAINT positive_amount CHECK (amount > 0)
   
   -- Query time: No validation overhead
   SELECT SUM(amount) FROM sales  -- Optimizer knows amount > 0
   ```
   **Advantages:**
   - **Write-once validation**: Avoid repeated constraint checks on reads
   - **Query optimization**: Query planner can use constraint information
   - **Read performance**: No runtime validation during queries

3. **Distributed System Design:**
   ```python
   # Validation parallelized across cluster nodes
   large_df.repartition(100) \
           .write.format('delta') \
           .mode('append') \
           .saveAsTable('employees')
   # Each partition validated independently on different nodes
   ```
   **Benefits:**
   - **Parallel validation**: Constraints checked across cluster nodes
   - **Scalability**: Validation scales with data volume
   - **Fault tolerance**: Failed partitions don't affect others

4. **Data Lake Architecture:**
   ```sql
   -- Multiple writers, single validation point
   INSERT INTO events SELECT * FROM stream_1  -- Writer 1
   INSERT INTO events SELECT * FROM stream_2  -- Writer 2  
   INSERT INTO events SELECT * FROM batch_3   -- Writer 3
   -- All writers subject to same constraints
   ```
   **Advantages:**
   - **Consistency across writers**: All ingestion paths validated equally
   - **Schema enforcement**: Prevents schema drift from multiple sources
   - **Quality gates**: Central quality control point

5. **Integration with Delta Lake Features:**
   ```sql
   -- Constraints work with all Delta operations
   UPDATE employees SET salary = salary * 1.1  -- CHECK constraint applied
   MERGE INTO employees USING updates ON ...   -- CHECK constraint applied
   COPY INTO employees FROM 's3://data/'       -- CHECK constraint applied
   ```
   **Benefits:**
   - **Universal enforcement**: Works with all write operations
   - **Streaming integration**: Real-time constraint validation
   - **Batch compatibility**: Same constraints for batch and streaming

**Write-Time vs Read-Time Comparison:**

| Approach | Pros | Cons |
|----------|------|------|
| **Write-Time** | Guaranteed data quality, query optimization, performance | Slower writes |
| **Read-Time** | Faster writes, flexible validation | Inconsistent data, query overhead |

Delta Lake chooses write-time enforcement for **data quality guarantees** over write performance.

**Q3: How do constraints contribute to data quality in a data lake?**

**Answer:**
Constraints are fundamental to maintaining data quality in data lakes:

1. **Data Integrity Foundation:**
   ```sql
   CREATE TABLE customer_profiles (
       customer_id INT NOT NULL,
       email STRING NOT NULL,
       age INT,
       signup_date DATE NOT NULL,
       CONSTRAINT valid_email CHECK (email LIKE '%@%.%'),
       CONSTRAINT reasonable_age CHECK (age >= 0 AND age <= 120),
       CONSTRAINT valid_signup CHECK (signup_date >= '2020-01-01')
   )
   ```
   **Quality Benefits:**
   - **Completeness**: NOT NULL ensures required data present
   - **Validity**: CHECK constraints enforce business rules
   - **Consistency**: Uniform data format and ranges

2. **Early Error Detection:**
   ```python
   # Bad data caught at ingestion time
   bad_customer_data = [
       (None, "invalid-email", 150, "1999-01-01"),  # Multiple violations
       (123, "valid@email.com", 25, "2023-01-01")   # Valid record
   ]
   
   try:
       df = spark.createDataFrame(bad_customer_data, schema)
       df.write.format('delta').mode('append').saveAsTable('customer_profiles')
   except Exception as e:
       print("Data quality issues caught before storage:")
       print("- NULL customer_id")
       print("- Invalid email format") 
       print("- Age out of range")
       print("- Signup date too old")
   ```
   **Benefits:**
   - **Fail fast**: Problems detected immediately
   - **Cost savings**: Avoid processing bad data downstream
   - **Debugging**: Clear error messages for data issues

3. **Downstream System Protection:**
   ```sql
   -- Analytics systems can rely on data quality
   CREATE VIEW customer_analytics AS
   SELECT 
       AVG(age) as avg_customer_age,        -- Safe: age always valid range
       COUNT(*) / COUNT(email) as email_rate -- Always 1.0: email never null
   FROM customer_profiles
   -- No need for defensive coding - constraints guarantee quality
   ```
   **Benefits:**
   - **Reliable analytics**: Downstream systems trust data quality
   - **Simplified code**: No defensive null checks required
   - **Accurate metrics**: Quality constraints ensure valid calculations

4. **Compliance and Governance:**
   ```sql
   -- GDPR compliance through constraints
   CREATE TABLE user_data (
       user_id STRING NOT NULL,
       email STRING NOT NULL,
       country_code STRING NOT NULL,
       data_processing_consent BOOLEAN NOT NULL,
       CONSTRAINT valid_country CHECK (country_code IN ('US', 'EU', 'UK', 'CA')),
       CONSTRAINT consent_required CHECK (data_processing_consent = true)
   )
   ```
   **Benefits:**
   - **Regulatory compliance**: Enforce legal requirements
   - **Audit trails**: Provable data quality standards
   - **Risk mitigation**: Prevent compliance violations

5. **Data Pipeline Reliability:**
   ```sql
   -- Multi-stage pipeline with quality gates
   CREATE TABLE bronze_events (
       event_id STRING NOT NULL,
       timestamp TIMESTAMP NOT NULL,
       user_id STRING NOT NULL,
       CONSTRAINT recent_events CHECK (timestamp >= current_date() - INTERVAL 30 DAYS)
   )
   
   CREATE TABLE silver_events (  
       event_id STRING NOT NULL,
       processed_timestamp TIMESTAMP NOT NULL,
       user_id STRING NOT NULL,
       event_type STRING NOT NULL,
       CONSTRAINT valid_event_type CHECK (event_type IN ('click', 'view', 'purchase'))
   )
   ```
   **Benefits:**
   - **Stage validation**: Quality gates at each processing stage
   - **Error isolation**: Problems caught before affecting downstream stages
   - **Pipeline reliability**: Consistent data flow guaranteed

6. **Business Rule Enforcement:**
   ```sql
   -- Business logic encoded as constraints
   CREATE TABLE orders (
       order_id STRING NOT NULL,
       customer_id STRING NOT NULL, 
       order_total DECIMAL(10,2) NOT NULL,
       discount_amount DECIMAL(10,2),
       tax_amount DECIMAL(10,2) NOT NULL,
       CONSTRAINT positive_total CHECK (order_total > 0),
       CONSTRAINT reasonable_discount CHECK (discount_amount <= order_total * 0.5),
       CONSTRAINT valid_tax CHECK (tax_amount >= 0 AND tax_amount <= order_total * 0.3)
   )
   ```
   **Benefits:**
   - **Business logic centralization**: Rules enforced at data layer
   - **Consistency**: Same rules applied regardless of data source
   - **Documentation**: Constraints serve as business rule documentation

**Data Quality Impact Measurement:**
```sql
-- Before constraints: Quality monitoring required
SELECT 
    COUNT(*) as total_records,
    COUNT(*) - COUNT(customer_id) as missing_customer_ids,
    COUNT(*) - COUNT(email) as missing_emails,
    SUM(CASE WHEN age < 0 OR age > 120 THEN 1 ELSE 0 END) as invalid_ages
FROM customer_data

-- After constraints: Quality guaranteed  
SELECT COUNT(*) as total_records,
       0 as data_quality_issues  -- Guaranteed by constraints
FROM customer_profiles
```

**Quality Benefits Summary:**
- **Prevention**: Stop bad data at ingestion
- **Trust**: Downstream systems can rely on data quality  
- **Efficiency**: Eliminate quality checks in every query
- **Compliance**: Meet regulatory data quality requirements
- **Documentation**: Constraints document quality expectations

---

## Exercise 6 Solutions: Advanced Constraint Scenarios

### Questions & Answers

**Q1: How do complex CHECK constraints help enforce business rules?**

**Answer:**
Complex CHECK constraints enable sophisticated business rule enforcement:

**Multi-Column Business Logic:**
```sql
CREATE TABLE employee_compensation (
    employee_id INT NOT NULL,
    base_salary DOUBLE NOT NULL,
    bonus_percentage DOUBLE,
    commission_rate DOUBLE,
    total_compensation DOUBLE,
    effective_date DATE NOT NULL,
    
    -- Complex business rules as constraints
    CONSTRAINT salary_bonus_relationship CHECK (
        total_compensation >= base_salary * (1 + COALESCE(bonus_percentage, 0) / 100)
    ),
    CONSTRAINT executive_compensation_limits CHECK (
        CASE 
            WHEN base_salary > 200000 THEN bonus_percentage <= 50
            ELSE true
        END
    ),
    CONSTRAINT commission_eligibility CHECK (
        CASE 
            WHEN commission_rate > 0 THEN base_salary >= 50000
            ELSE true
        END
    )
)
```

**Business Rule Benefits:**

1. **Logical Consistency Enforcement:**
   ```sql
   -- Ensures total compensation always makes mathematical sense
   CONSTRAINT logical_compensation CHECK (
       total_compensation >= base_salary + 
                           COALESCE(base_salary * bonus_percentage / 100, 0)
   )
   ```
   **Prevents:**
   - Total compensation less than base salary
   - Inconsistent bonus calculations
   - Mathematical errors in compensation data

2. **Conditional Business Rules:**
   ```sql
   -- Different rules for different employee categories
   CONSTRAINT category_based_rules CHECK (
       CASE 
           WHEN job_level = 'Executive' THEN 
               base_salary >= 150000 AND bonus_percentage <= 100
           WHEN job_level = 'Manager' THEN 
               base_salary >= 80000 AND bonus_percentage <= 50
           WHEN job_level = 'Individual Contributor' THEN 
               base_salary >= 40000 AND bonus_percentage <= 25
           ELSE true
       END
   )
   ```
   **Benefits:**
   - **Role-appropriate rules**: Different constraints per job level
   - **Business hierarchy**: Enforce organizational compensation structure
   - **Compliance**: Meet regulatory compensation guidelines

3. **Cross-Column Validation:**
   ```sql
   -- Ensure related fields are consistent
   CONSTRAINT sales_role_commission CHECK (
       CASE 
           WHEN department = 'Sales' THEN commission_rate IS NOT NULL
           WHEN department != 'Sales' THEN commission_rate IS NULL OR commission_rate = 0
           ELSE true
       END
   )
   ```
   **Enforces:**
   - Sales employees must have commission structure
   - Non-sales employees cannot earn commission
   - Department-specific compensation models

4. **Temporal Business Rules:**
   ```sql
   CONSTRAINT compensation_change_limits CHECK (
       CASE 
           WHEN effective_date > '2023-01-01' THEN 
               total_compensation <= base_salary * 3  -- Recent hire limits
           ELSE true  -- Grandfathered employees exempt
       END
   )
   ```
   **Capabilities:**
   - **Time-based rules**: Different rules for different periods
   - **Grandfathering**: Exempt existing records from new rules
   - **Regulatory changes**: Implement new compliance requirements

**Real-World Business Rule Examples:**
```sql
-- Financial services compliance
CREATE TABLE loan_applications (
    application_id STRING NOT NULL,
    loan_amount DECIMAL(12,2) NOT NULL,
    annual_income DECIMAL(12,2) NOT NULL,
    debt_to_income_ratio DECIMAL(5,4),
    credit_score INT,
    loan_purpose STRING NOT NULL,
    
    -- Regulatory compliance constraints
    CONSTRAINT debt_to_income_limit CHECK (
        debt_to_income_ratio <= 0.43  -- Federal regulation
    ),
    CONSTRAINT loan_to_income_ratio CHECK (
        loan_amount <= annual_income * 5  -- Internal risk policy
    ),
    CONSTRAINT credit_score_requirements CHECK (
        CASE loan_purpose
            WHEN 'mortgage' THEN credit_score >= 620
            WHEN 'auto' THEN credit_score >= 580
            WHEN 'personal' THEN credit_score >= 640
            ELSE credit_score >= 500
        END
    )
)
```

**Q2: What happens when multiple constraints are violated simultaneously?**

**Answer:**
When multiple constraints are violated, Delta Lake's behavior is specific and predictable:

**Constraint Evaluation Order:**
```python
# Test data violating multiple constraints
test_data = [(
    1,          # employee_id: OK
    -50000.0,   # base_salary: ❌ Violates positive_base_salary CHECK
    150.0,      # bonus_percentage: ❌ Violates reasonable_bonus CHECK (>100)
    60.0,       # commission_rate: ❌ Violates reasonable_commission CHECK (>50)
    -40000.0,   # total_compensation: ❌ Violates logical_compensation CHECK 
    "2019-01-01"  # effective_date: ❌ Violates future_effective_date CHECK
)]
```

**Error Behavior:**

1. **First Violation Reported:**
```python
try:
    df.write.format('delta').mode('append').saveAsTable('employee_compensation')
except Exception as e:
    print("Delta Lake reports first constraint violation encountered:")
    print(e)
    # Output: "CHECK constraint positive_base_salary violated by row..."
    # Other violations not reported until first is fixed
```

**Sequential Constraint Fixing:**
```python
# Fix constraints one by one
test_cases = [
    # Fix 1: Positive base salary
    (1, 50000.0, 150.0, 60.0, -40000.0, "2019-01-01"),
    # Next error: "CHECK constraint reasonable_bonus violated..."
    
    # Fix 2: Reasonable bonus  
    (1, 50000.0, 50.0, 60.0, -40000.0, "2019-01-01"),
    # Next error: "CHECK constraint reasonable_commission violated..."
    
    # Fix 3: Reasonable commission
    (1, 50000.0, 50.0, 25.0, -40000.0, "2019-01-01"),
    # Next error: "CHECK constraint logical_compensation violated..."
    
    # Fix 4: Logical compensation
    (1, 50000.0, 50.0, 25.0, 75000.0, "2019-01-01"),
    # Next error: "CHECK constraint future_effective_date violated..."
    
    # Fix 5: Valid effective date
    (1, 50000.0, 50.0, 25.0, 75000.0, "2023-01-01"),
    # Success: All constraints satisfied
]
```

2. **Batch Behavior:**
```python
# Multiple rows: First violation in batch stops entire operation
mixed_batch = [
    (1, 60000.0, 10.0, 5.0, 66000.0, "2023-01-01"),  # ✅ Valid
    (2, -30000.0, 5.0, 2.0, -28500.0, "2023-02-01"),  # ❌ Invalid salary
    (3, 80000.0, 15.0, 8.0, 92000.0, "2023-03-01"),  # ✅ Valid (never processed)
]
# Result: NO rows inserted - entire batch fails
```

3. **Multiple Constraint Names in Error:**
```sql
-- Some constraint violations may mention related constraints
ALTER TABLE test_table ADD CONSTRAINT rule1 CHECK (a > 0)
ALTER TABLE test_table ADD CONSTRAINT rule2 CHECK (b > a)

-- Insert violates both: a = -1, b = -2  
-- Error may reference both rule1 and rule2 if they're interdependent
```

**Best Practices for Multiple Constraints:**

1. **Constraint Testing Strategy:**
```python
def test_constraints_individually():
    """Test each constraint violation separately for debugging"""
    
    base_valid_record = (1, 60000.0, 10.0, 5.0, 66000.0, "2023-01-01")
    
    constraint_tests = [
        # Test each constraint individually
        (1, -1000.0, 10.0, 5.0, 66000.0, "2023-01-01"),   # salary constraint
        (1, 60000.0, 200.0, 5.0, 66000.0, "2023-01-01"),  # bonus constraint  
        (1, 60000.0, 10.0, 100.0, 66000.0, "2023-01-01"), # commission constraint
        (1, 60000.0, 10.0, 5.0, 50000.0, "2023-01-01"),   # total comp constraint
        (1, 60000.0, 10.0, 5.0, 66000.0, "2018-01-01"),   # date constraint
    ]
    
    for test_record in constraint_tests:
        try:
            test_df = spark.createDataFrame([test_record], schema)
            test_df.write.format('delta').mode('overwrite').saveAsTable('temp_test')
            print(f"✅ Constraint test passed: {test_record}")
        except Exception as e:
            print(f"❌ Constraint violation: {str(e)[:100]}...")
```

2. **Comprehensive Data Validation:**
```python
def validate_before_insert(df):
    """Pre-validate data against known constraints to identify all issues"""
    
    issues = []
    
    # Check each constraint manually
    negative_salary = df.filter(col("base_salary") <= 0).count()
    if negative_salary > 0:
        issues.append(f"{negative_salary} records with negative/zero salary")
    
    invalid_bonus = df.filter(col("bonus_percentage") > 100).count() 
    if invalid_bonus > 0:
        issues.append(f"{invalid_bonus} records with bonus > 100%")
    
    # Report all issues before attempting insert
    if issues:
        print("Data quality issues found:")
        for issue in issues:
            print(f"  - {issue}")
        return False
    return True
```

**Practical Impact:**
- **Development**: Fix constraints one at a time during testing
- **Production**: Implement comprehensive data validation before writes  
- **Monitoring**: Alert on constraint violations to catch data quality issues
- **Documentation**: Clearly document constraint dependencies

**Q3: How would you design constraints for a real-world scenario?**

**Answer:**
Designing effective constraints requires balancing data quality, performance, and business requirements:

**Real-World Scenario: E-commerce Order Management System**

**1. Requirement Analysis:**
```sql
-- Business Requirements Analysis:
-- - Orders must have valid customers and products
-- - Order amounts must be positive and reasonable  
-- - Discounts cannot exceed order value
-- - Shipping addresses are required for physical products
-- - Order dates must be reasonable (not future, not too old)
-- - Tax calculations must be within expected ranges
-- - Order status follows valid state transitions
```

**2. Hierarchical Constraint Design:**

**Core Data Integrity (NOT NULL):**
```sql
CREATE TABLE ecommerce_orders (
    order_id STRING NOT NULL,           -- Always required
    customer_id STRING NOT NULL,        -- Always required  
    order_date DATE NOT NULL,           -- Always required
    order_status STRING NOT NULL,       -- Always required
    subtotal DECIMAL(12,2) NOT NULL,    -- Always required
    tax_amount DECIMAL(12,2) NOT NULL,  -- Always required
    total_amount DECIMAL(12,2) NOT NULL -- Always required
    -- ... other columns
)
```

**Business Logic Constraints (CHECK):**
```sql
-- Level 1: Basic validity checks
ALTER TABLE ecommerce_orders ADD CONSTRAINT positive_amounts 
CHECK (subtotal >= 0 AND tax_amount >= 0 AND total_amount >= 0);

ALTER TABLE ecommerce_orders ADD CONSTRAINT reasonable_order_dates
CHECK (order_date >= '2020-01-01' AND order_date <= CURRENT_DATE() + INTERVAL 1 DAY);

-- Level 2: Mathematical consistency  
ALTER TABLE ecommerce_orders ADD CONSTRAINT amount_consistency
CHECK (total_amount >= subtotal AND total_amount <= subtotal + tax_amount + shipping_fee);

-- Level 3: Business rule enforcement
ALTER TABLE ecommerce_orders ADD CONSTRAINT valid_order_status
CHECK (order_status IN ('pending', 'confirmed', 'shipped', 'delivered', 'cancelled', 'returned'));

ALTER TABLE ecommerce_orders ADD CONSTRAINT reasonable_discount
CHECK (discount_amount IS NULL OR (discount_amount >= 0 AND discount_amount <= subtotal * 0.9));

-- Level 4: Complex business scenarios
ALTER TABLE ecommerce_orders ADD CONSTRAINT shipping_logic
CHECK (
    CASE 
        WHEN product_type = 'digital' THEN shipping_fee = 0 AND shipping_address IS NULL
        WHEN product_type = 'physical' THEN shipping_fee >= 0 AND shipping_address IS NOT NULL
        ELSE true
    END
);
```

**3. Performance-Conscious Design:**

**Simple vs Complex Constraints:**
```sql
-- ✅ Good: Simple, fast constraints
ALTER TABLE ecommerce_orders ADD CONSTRAINT order_id_format
CHECK (order_id LIKE 'ORD-%' AND LENGTH(order_id) = 15);

-- ⚠️ Careful: More expensive constraints
ALTER TABLE ecommerce_orders ADD CONSTRAINT tax_rate_validation  
CHECK (
    CASE 
        WHEN shipping_state IN ('CA', 'NY', 'TX') THEN 
            tax_amount BETWEEN subtotal * 0.06 AND subtotal * 0.11
        WHEN shipping_state IN ('OR', 'NH', 'MT') THEN
            tax_amount = 0
        ELSE 
            tax_amount BETWEEN 0 AND subtotal * 0.15
    END
);

-- ❌ Avoid: Subquery constraints (very expensive)
-- CHECK (customer_id IN (SELECT customer_id FROM customers WHERE status = 'active'))
```

**4. Constraint Evolution Strategy:**

**Phase 1: Basic Constraints (Launch)**
```sql
-- Start with essential constraints only
CREATE TABLE ecommerce_orders_v1 (
    order_id STRING NOT NULL,
    customer_id STRING NOT NULL,
    total_amount DECIMAL(12,2) NOT NULL,
    CONSTRAINT positive_total CHECK (total_amount > 0),
    CONSTRAINT valid_order_id CHECK (order_id LIKE 'ORD-%')
);
```

**Phase 2: Enhanced Validation (Post-Launch)**
```sql
-- Add business rule constraints after system stabilizes
ALTER TABLE ecommerce_orders ADD CONSTRAINT status_validation
CHECK (order_status IN ('pending', 'confirmed', 'shipped', 'delivered'));

ALTER TABLE ecommerce_orders ADD CONSTRAINT discount_limits  
CHECK (discount_amount <= subtotal * 0.5);
```

**Phase 3: Advanced Business Logic (Mature System)**
```sql
-- Add complex constraints after understanding data patterns
ALTER TABLE ecommerce_orders ADD CONSTRAINT shipping_business_rules
CHECK (
    CASE shipping_method
        WHEN 'overnight' THEN shipping_fee >= 15.00
        WHEN 'express' THEN shipping_fee >= 8.00  
        WHEN 'standard' THEN shipping_fee >= 0.00
        ELSE true
    END
);
```

**5. Error Handling and User Experience:**

**Meaningful Constraint Names:**
```sql
-- ✅ Good: Descriptive constraint names
ALTER TABLE ecommerce_orders ADD CONSTRAINT order_amount_positive 
CHECK (total_amount > 0);

ALTER TABLE ecommerce_orders ADD CONSTRAINT discount_not_exceeds_subtotal
CHECK (discount_amount <= subtotal);

-- ❌ Poor: Generic constraint names  
ALTER TABLE ecommerce_orders ADD CONSTRAINT chk_1 CHECK (total_amount > 0);
ALTER TABLE ecommerce_orders ADD CONSTRAINT chk_2 CHECK (discount_amount <= subtotal);
```

**6. Testing and Validation Strategy:**

**Comprehensive Test Suite:**
```python
def test_order_constraints():
    """Comprehensive constraint testing for e-commerce orders"""
    
    # Valid baseline order
    valid_order = {
        "order_id": "ORD-2023-001234",
        "customer_id": "CUST-567890",
        "order_date": "2023-10-15",
        "subtotal": 100.00,
        "tax_amount": 8.50,
        "shipping_fee": 5.99,
        "discount_amount": 10.00,
        "total_amount": 104.49,
        "order_status": "pending"
    }
    
    # Test each constraint systematically
    constraint_tests = [
        # Positive amounts
        {**valid_order, "total_amount": -50.00},  # Should fail
        
        # Date validation
        {**valid_order, "order_date": "2025-12-31"},  # Should fail (future)
        {**valid_order, "order_date": "2019-01-01"},  # Should fail (too old)
        
        # Amount consistency
        {**valid_order, "total_amount": 50.00},  # Should fail (less than subtotal)
        
        # Valid status
        {**valid_order, "order_status": "invalid_status"},  # Should fail
        
        # Discount validation  
        {**valid_order, "discount_amount": 150.00},  # Should fail (exceeds subtotal)
    ]
    
    return run_constraint_tests(constraint_tests)
```

**7. Monitoring and Maintenance:**

**Constraint Monitoring Dashboard:**
```sql
CREATE VIEW constraint_monitoring AS
SELECT 
    'ecommerce_orders' as table_name,
    COUNT(*) as total_records,
    COUNT(CASE WHEN total_amount <= 0 THEN 1 END) as negative_amounts,
    COUNT(CASE WHEN order_date > CURRENT_DATE() THEN 1 END) as future_orders,
    COUNT(CASE WHEN discount_amount > subtotal THEN 1 END) as invalid_discounts,
    MAX(order_date) as latest_order_date,
    MIN(order_date) as earliest_order_date,
    CURRENT_TIMESTAMP() as check_timestamp
FROM ecommerce_orders;
```

**Design Principles Summary:**
1. **Start Simple**: Begin with essential constraints, add complexity gradually
2. **Business Alignment**: Constraints should reflect actual business rules
3. **Performance Balance**: Consider constraint evaluation cost vs. benefit
4. **Clear Naming**: Use descriptive constraint names for debugging
5. **Systematic Testing**: Test each constraint individually and in combination
6. **Evolution Strategy**: Plan for constraint changes as business evolves
7. **Monitoring**: Continuously monitor constraint violations and data quality

This approach ensures constraints effectively enforce data quality while maintaining system performance and operational flexibility.

---

## Exercise 7 Solutions: Data Governance and Best Practices

### Questions & Answers

**Q1: How do table properties contribute to data governance?**

**Answer:**
Table properties are critical metadata that enables comprehensive data governance:

**1. Data Classification and Security:**
```sql
ALTER TABLE hr_analytics_lab.hr_data.employees_managed 
SET TBLPROPERTIES (
    'data_classification' = 'PII',
    'sensitivity_level' = 'high',
    'encryption_required' = 'true',
    'access_control_required' = 'rbac'
);
```
**Governance Benefits:**
- **Automated Security**: Security tools can automatically apply encryption and access controls
- **Compliance Scanning**: Automated discovery of sensitive data for GDPR, HIPAA compliance
- **Risk Assessment**: Identify high-risk tables requiring additional protection
- **Audit Preparation**: Quickly locate all PII data for regulatory audits

**2. Data Ownership and Stewardship:**
```sql
ALTER TABLE sales_data.customer_orders
SET TBLPROPERTIES (
    'data_steward' = 'sales_analytics_team',
    'business_owner' = 'chief_revenue_officer',  
    'technical_contact' = 'data_engineering@company.com',
    'created_by' = 'etl_pipeline_v2.1',
    'cost_center' = 'sales_department'
);
```
**Governance Benefits:**
- **Accountability**: Clear ownership chain for data issues
- **Contact Information**: Quick resolution of data problems
- **Cost Allocation**: Charge storage and compute costs to appropriate departments
- **Change Management**: Know who to contact before modifying table structure

**3. Data Quality and Reliability:**
```sql
ALTER TABLE analytics.customer_360_view
SET TBLPROPERTIES (
    'quality_score' = '94.5',
    'last_quality_check' = '2023-10-15',
    'data_freshness' = 'daily',
    'sla_tier' = 'gold',
    'quality_rules' = 'completeness>95%,accuracy>90%,consistency>98%'
);
```
**Governance Benefits:**
- **Quality Monitoring**: Automated alerts when quality scores drop
- **SLA Management**: Different service levels for different business importance
- **Trust Indicators**: Users can assess data reliability before use
- **Quality Tracking**: Historical quality trends for improvement initiatives

**4. Regulatory and Compliance Metadata:**
```sql
ALTER TABLE finance.transaction_records  
SET TBLPROPERTIES (
    'retention_policy' = '7_years',
    'retention_start_date' = '2023-01-01',
    'regulatory_framework' = 'SOX,PCI_DSS',
    'audit_required' = 'quarterly',
    'deletion_allowed' = 'false',
    'right_to_be_forgotten' = 'customer_id_based'
);
```
**Governance Benefits:**
- **Automated Retention**: Policies enforced by data lifecycle management tools
- **Compliance Reporting**: Generate compliance status reports automatically  
- **Legal Hold**: Prevent data deletion during legal proceedings
- **GDPR Compliance**: Support right to be forgotten requests

**5. Operational Metadata:**
```sql
ALTER TABLE streaming.event_logs
SET TBLPROPERTIES (
    'update_frequency' = 'real_time',
    'source_system' = 'kafka_cluster_prod',
    'downstream_dependencies' = 'dashboard_daily,ml_model_training',
    'backup_schedule' = 'hourly',
    'disaster_recovery_rto' = '15_minutes'
);
```
**Governance Benefits:**
- **Impact Analysis**: Understand downstream effects of changes
- **Capacity Planning**: Right-size infrastructure based on usage patterns
- **Incident Response**: Faster troubleshooting with operational context
- **Change Coordination**: Coordinate changes across dependent systems

**Q2: What metadata should you track for production tables?**

**Answer:**
Production tables require comprehensive metadata tracking across multiple dimensions:

**Essential Production Metadata Categories:**

**1. Identity and Classification:**
```sql
-- Core identification metadata
'table_name' = 'customer_transactions',
'table_purpose' = 'Financial transaction records for customer billing',
'business_domain' = 'finance',
'data_classification' = 'confidential_financial',
'criticality_tier' = 'tier_1_critical',
'table_type' = 'fact_table'
```

**2. Ownership and Contacts:**
```sql  
-- Accountability metadata
'data_steward' = 'finance_data_team',
'business_owner' = 'cfo_organization', 
'technical_owner' = 'data_platform_team',
'on_call_contact' = 'data_engineering_oncall@company.com',
'escalation_path' = 'data_lead->engineering_manager->vp_engineering'
```

**3. Data Quality and SLA:**
```sql
-- Quality and reliability metadata
'sla_tier' = 'platinum',  -- platinum/gold/silver/bronze
'availability_sla' = '99.9%',
'freshness_sla' = '15_minutes',
'quality_score_threshold' = '95%',
'completeness_sla' = '99%',
'accuracy_sla' = '98%',
'last_quality_check' = '2023-10-15T14:30:00Z',
'quality_check_frequency' = 'hourly'
```

**4. Security and Compliance:**
```sql
-- Security and regulatory metadata  
'contains_pii' = 'true',
'pii_columns' = 'customer_email,customer_phone,billing_address',
'encryption_at_rest' = 'aes_256',
'encryption_in_transit' = 'tls_1.3',
'access_control_model' = 'rbac_with_abac',
'regulatory_frameworks' = 'SOX,PCI_DSS,GDPR',
'data_residency_requirements' = 'US_only',
'retention_period' = '2555_days'  -- 7 years
```

**5. Operational Characteristics:**
```sql
-- Operational metadata
'source_system' = 'payment_processing_api',
'ingestion_pattern' = 'streaming_kafka',
'update_frequency' = 'real_time',
'batch_size' = '10000_records_per_minute',
'peak_usage_hours' = '09:00-17:00_EST',
'backup_frequency' = 'hourly',
'backup_retention' = '90_days',
'disaster_recovery_rpo' = '5_minutes',
'disaster_recovery_rto' = '30_minutes'
```

**6. Dependencies and Lineage:**
```sql  
-- Dependency metadata
'upstream_tables' = 'raw_events.payment_events,reference.customer_master',
'downstream_tables' = 'analytics.daily_revenue,ml.fraud_detection_features',
'dependent_dashboards' = 'finance_dashboard,executive_kpis',
'dependent_ml_models' = 'fraud_detection_v2,churn_prediction_v3',
'api_consumers' = 'billing_service,customer_portal'
```

**7. Performance and Optimization:**
```sql
-- Performance metadata
'partition_strategy' = 'date_based_monthly',
'optimization_schedule' = 'weekly_sunday_02:00',
'vacuum_schedule' = 'weekly_sunday_04:00', 
'expected_daily_growth' = '50_gb',
'query_patterns' = 'mostly_recent_data_90_days',
'indexing_strategy' = 'bloom_filter_on_customer_id'
```

**8. Change Management:**
```sql
-- Change tracking metadata
'schema_version' = 'v2.3',
'last_schema_change' = '2023-09-15T10:00:00Z',
'schema_change_frequency' = 'monthly',
'breaking_change_notification_period' = '30_days',
'change_approval_required' = 'true',
'change_approval_group' = 'data_architecture_board'
```

**Metadata Implementation Strategy:**

**1. Automated Metadata Collection:**
```python
def collect_table_metadata(table_name):
    """Automated collection of operational metadata"""
    
    # Collect from system tables
    details = spark.sql(f"DESCRIBE DETAIL {table_name}").collect()[0]
    history = spark.sql(f"DESCRIBE HISTORY {table_name}").collect()
    
    auto_metadata = {
        'current_size_bytes': details['sizeInBytes'],
        'num_files': details['numFiles'], 
        'last_modified': details['lastModified'],
        'version_count': len(history),
        'creation_date': min(h['timestamp'] for h in history),
        'last_write_operation': history[0]['operation'] if history else 'unknown'
    }
    
    return auto_metadata
```

**2. Metadata Validation and Monitoring:**
```sql
-- Create metadata validation view
CREATE VIEW data_governance.metadata_completeness AS
SELECT 
    table_catalog,
    table_schema, 
    table_name,
    CASE WHEN table_properties['data_steward'] IS NOT NULL THEN 1 ELSE 0 END as has_steward,
    CASE WHEN table_properties['sla_tier'] IS NOT NULL THEN 1 ELSE 0 END as has_sla,
    CASE WHEN table_properties['data_classification'] IS NOT NULL THEN 1 ELSE 0 END as has_classification,
    CASE WHEN table_properties['retention_policy'] IS NOT NULL THEN 1 ELSE 0 END as has_retention,
    (has_steward + has_sla + has_classification + has_retention) / 4.0 as metadata_completeness_score
FROM information_schema.tables
WHERE table_type = 'MANAGED'
```

**3. Metadata Governance Dashboard:**
```sql
-- Production readiness assessment
CREATE VIEW data_governance.production_readiness AS
SELECT 
    table_name,
    CASE 
        WHEN metadata_completeness_score >= 0.9 THEN 'Production Ready'
        WHEN metadata_completeness_score >= 0.7 THEN 'Needs Minor Updates'
        WHEN metadata_completeness_score >= 0.5 THEN 'Needs Major Updates'
        ELSE 'Not Production Ready'
    END as readiness_status,
    CASE WHEN table_properties['last_quality_check'] > CURRENT_TIMESTAMP() - INTERVAL 24 HOURS 
         THEN 'Recent' ELSE 'Outdated' END as quality_check_status
FROM data_governance.metadata_completeness
```

**Q3: How do Unity Catalog constraints support compliance requirements?**

**Answer:**
Unity Catalog constraints provide foundational support for regulatory compliance:

**1. GDPR (General Data Protection Regulation) Support:**

**Right to Data Quality:**
```sql
-- Ensure personal data quality through constraints
CREATE TABLE customer_profiles (
    customer_id STRING NOT NULL,
    email STRING NOT NULL,
    phone STRING,
    birth_date DATE,
    country STRING NOT NULL,
    
    -- GDPR Article 5: Data quality principles
    CONSTRAINT valid_email_format CHECK (email LIKE '%@%.%'),
    CONSTRAINT reasonable_birth_date CHECK (birth_date >= '1900-01-01' AND birth_date <= CURRENT_DATE()),
    CONSTRAINT valid_country_code CHECK (LENGTH(country) = 2),
    
    -- Metadata for GDPR compliance
    consent_timestamp TIMESTAMP NOT NULL,
    data_processing_purpose STRING NOT NULL,
    
    CONSTRAINT valid_consent CHECK (consent_timestamp <= CURRENT_TIMESTAMP()),
    CONSTRAINT valid_purpose CHECK (data_processing_purpose IN (
        'service_delivery', 'marketing_consent', 'legal_obligation', 'legitimate_interest'
    ))
) 
TBLPROPERTIES (
    'contains_pii' = 'true',
    'gdpr_lawful_basis' = 'consent,contract',
    'retention_period' = '2555_days',  -- 7 years
    'deletion_on_request' = 'customer_id_based'
);
```

**Right to be Forgotten Implementation:**
```sql
-- Constraints supporting data subject rights
CREATE TABLE data_deletion_requests (
    request_id STRING NOT NULL,
    customer_id STRING NOT NULL,
    request_date DATE NOT NULL,
    request_type STRING NOT NULL,
    processing_status STRING NOT NULL,
    
    -- Ensure proper deletion workflow
    CONSTRAINT valid_request_type CHECK (request_type IN ('deletion', 'anonymization', 'data_export')),
    CONSTRAINT valid_status CHECK (processing_status IN ('pending', 'in_progress', 'completed', 'rejected')),
    CONSTRAINT recent_request CHECK (request_date >= CURRENT_DATE() - INTERVAL 1000 DAYS),
    
    -- Compliance timing requirements
    completion_deadline DATE,
    CONSTRAINT gdpr_deadline CHECK (
        CASE WHEN request_type = 'deletion' 
             THEN completion_deadline <= request_date + INTERVAL 30 DAYS
             ELSE true END
    )
);
```

**2. SOX (Sarbanes-Oxley) Compliance:**

**Financial Data Integrity:**
```sql
-- SOX Section 404: Internal controls over financial reporting
CREATE TABLE financial_transactions (
    transaction_id STRING NOT NULL,
    account_id STRING NOT NULL,
    transaction_date DATE NOT NULL,
    amount DECIMAL(15,2) NOT NULL,
    transaction_type STRING NOT NULL,
    created_by STRING NOT NULL,
    approved_by STRING,
    audit_trail_id STRING NOT NULL,
    
    -- SOX data integrity constraints
    CONSTRAINT positive_amounts CHECK (ABS(amount) > 0),
    CONSTRAINT valid_transaction_types CHECK (
        transaction_type IN ('debit', 'credit', 'adjustment', 'reversal')
    ),
    CONSTRAINT recent_transactions CHECK (
        transaction_date >= '2020-01-01' AND 
        transaction_date <= CURRENT_DATE() + INTERVAL 1 DAY
    ),
    
    -- Segregation of duties
    CONSTRAINT segregation_of_duties CHECK (
        CASE WHEN ABS(amount) > 10000 
             THEN approved_by IS NOT NULL AND approved_by != created_by
             ELSE true END
    ),
    
    -- Audit trail requirements
    CONSTRAINT audit_trail_required CHECK (audit_trail_id IS NOT NULL)
) 
TBLPROPERTIES (
    'sox_control' = 'ITGC_001',
    'retention_period' = '2555_days',  -- 7 years minimum
    'audit_frequency' = 'quarterly',
    'deletion_prohibited' = 'true'
);
```

**3. HIPAA (Health Insurance Portability and Accountability Act):**

**Protected Health Information (PHI):**
```sql
-- HIPAA minimum necessary standard
CREATE TABLE patient_records (
    patient_id STRING NOT NULL,
    medical_record_number STRING NOT NULL,
    diagnosis_date DATE NOT NULL,
    treatment_code STRING NOT NULL,
    provider_id STRING NOT NULL,
    
    -- HIPAA data quality requirements
    CONSTRAINT valid_medical_record CHECK (LENGTH(medical_record_number) >= 8),
    CONSTRAINT reasonable_diagnosis_date CHECK (
        diagnosis_date >= '1900-01-01' AND 
        diagnosis_date <= CURRENT_DATE()
    ),
    CONSTRAINT valid_treatment_code CHECK (treatment_code LIKE 'ICD-%'),
    
    -- Access control metadata
    last_accessed_by STRING,
    last_access_timestamp TIMESTAMP,
    access_purpose STRING,
    
    CONSTRAINT valid_access_purpose CHECK (
        access_purpose IN ('treatment', 'payment', 'operations', 'research_approved')
    )
)
TBLPROPERTIES (
    'contains_phi' = 'true',
    'hipaa_covered_entity' = 'true',
    'minimum_necessary_applied' = 'true',
    'breach_notification_required' = 'true',
    'retention_period' = '2190_days'  -- 6 years
);
```

**4. PCI DSS (Payment Card Industry Data Security Standard):**

**Cardholder Data Protection:**
```sql
-- PCI DSS Requirement 3: Protect stored cardholder data
CREATE TABLE payment_transactions (
    transaction_id STRING NOT NULL,
    merchant_id STRING NOT NULL,
    transaction_amount DECIMAL(10,2) NOT NULL,
    currency_code STRING NOT NULL,
    transaction_timestamp TIMESTAMP NOT NULL,
    
    -- PCI DSS data protection constraints  
    card_token STRING NOT NULL,  -- Tokenized, not actual card number
    last_four_digits STRING,
    
    -- No storage of prohibited data (enforced by constraints)
    CONSTRAINT no_full_pan CHECK (card_token NOT LIKE '%[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]%'),
    CONSTRAINT valid_last_four CHECK (last_four_digits IS NULL OR 
        (LENGTH(last_four_digits) = 4 AND last_four_digits REGEXP '^[0-9]{4}$')),
    
    -- Transaction validation
    CONSTRAINT positive_amount CHECK (transaction_amount > 0),
    CONSTRAINT valid_currency CHECK (LENGTH(currency_code) = 3),
    CONSTRAINT recent_transaction CHECK (
        transaction_timestamp >= CURRENT_TIMESTAMP() - INTERVAL 30 DAYS
    )
)
TBLPROPERTIES (
    'pci_dss_scope' = 'true',
    'cardholder_data_present' = 'tokenized_only',
    'encryption_required' = 'aes_256',
    'access_logging_required' = 'true',
    'retention_period' = '365_days'
);
```

**5. Cross-Regulatory Constraint Patterns:**

**Universal Compliance Patterns:**
```sql
-- Pattern for audit trail tables
CREATE TABLE audit_log_template (
    event_id STRING NOT NULL,
    table_name STRING NOT NULL,
    operation_type STRING NOT NULL,
    user_id STRING NOT NULL,
    event_timestamp TIMESTAMP NOT NULL,
    before_values STRING,  -- JSON of old values
    after_values STRING,   -- JSON of new values
    
    -- Universal audit constraints
    CONSTRAINT valid_operation CHECK (
        operation_type IN ('INSERT', 'UPDATE', 'DELETE', 'SELECT')
    ),
    CONSTRAINT recent_event CHECK (
        event_timestamp <= CURRENT_TIMESTAMP() AND
        event_timestamp >= CURRENT_TIMESTAMP() - INTERVAL 10 YEARS
    ),
    CONSTRAINT audit_completeness CHECK (
        CASE operation_type
            WHEN 'INSERT' THEN after_values IS NOT NULL
            WHEN 'UPDATE' THEN before_values IS NOT NULL AND after_values IS NOT NULL  
            WHEN 'DELETE' THEN before_values IS NOT NULL
            ELSE true
        END
    )
)
TBLPROPERTIES (
    'audit_table' = 'true',
    'deletion_prohibited' = 'true',
    'retention_period' = '3650_days',  -- 10 years
    'compliance_frameworks' = 'SOX,GDPR,HIPAA,PCI_DSS'
);
```

**Constraint-Based Compliance Benefits:**

1. **Automated Enforcement:** Constraints automatically prevent non-compliant data
2. **Audit Evidence:** Constraint violations provide audit trail of compliance attempts  
3. **Risk Mitigation:** Technical controls reduce human error in compliance
4. **Documentation:** Constraints serve as machine-readable compliance documentation
5. **Continuous Monitoring:** Constraint violations can trigger compliance alerts
6. **Data Quality:** Ensures compliance data meets regulatory quality standards

**Implementation Best Practices:**

```sql
-- Compliance metadata template
ALTER TABLE compliance_critical_table SET TBLPROPERTIES (
    'compliance_frameworks' = 'GDPR,SOX,HIPAA',
    'compliance_owner' = 'chief_compliance_officer',
    'last_compliance_audit' = '2023-09-15',
    'next_compliance_audit' = '2024-03-15',
    'compliance_status' = 'compliant',
    'automated_compliance_checks' = 'enabled',
    'manual_review_required' = 'quarterly'
);
```

This comprehensive approach ensures that Unity Catalog constraints not only enforce data quality but also provide the technical foundation for regulatory compliance across multiple frameworks.

---

## Summary

This solution guide provides comprehensive answers to all lab questions, covering:

1. **Unity Catalog Fundamentals**: Managed vs external tables, storage, and governance
2. **External Locations**: Security, compliance, and architecture benefits  
3. **View Types**: Temporary, global temporary, and stored views for different use cases
4. **CTAS Operations**: Performance, complexity, and design considerations
5. **Delta Constraints**: Enforced vs metadata-only constraints and their purposes
6. **Advanced Scenarios**: Complex business rules and multi-constraint handling
7. **Data Governance**: Metadata management and compliance support

Each solution includes practical examples, best practices, and real-world implementation guidance to help students master Unity Catalog and Delta Lake constraint concepts for production environments.